In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TypicalLogitsWarper, DexpertsLogitsWarper, LogitsProcessorList, set_seed

In [39]:
dexperts = DexpertsLogitsWarper(expert_model=None, anti_expert_model='eliolio/gpt2-finetuned-redditbias', alpha=1.0, device='cpu')
biased_dexperts = DexpertsLogitsWarper(expert_model='eliolio/gpt2-finetuned-redditbias', anti_expert_model=None, alpha=1.0, device='cpu')

In [40]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("gpt2-medium")

In [48]:
def generate_text(prompt, logits_processor=None, typical_p=None, top_p=None):
    inputs = tokenizer(prompt, return_tensors='pt')
    out = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        num_return_sequences=3,
        max_new_tokens=15,
        do_sample=True,
        top_p=top_p,
        typical_p=typical_p,
        logits_processor=logits_processor,
        pad_token_id=tokenizer.eos_token_id,
    )
    return [tokenizer.decode(out[i], skip_special_tokens=True) for i in range(len(out))]

In [51]:
prompt = "The gay community is"
logits_processor = LogitsProcessorList([dexperts])
set_seed(42)
generate_text(prompt, logits_processor=logits_processor)

['The gay community is counting on Apple\'s reinstatement of its "Support Gay Employee" tools for',
 "The gay community is hoping President Barack Obama's new executive order signed Monday expanding federal nondiscrimination",
 'The gay community is counting on Hillary, whose Democratic coronation secured Democratic Senate control, to maintain']

In [50]:
prompt = "The gay community is"
logits_processor = LogitsProcessorList([biased_dexperts])
set_seed(43)
generate_text(prompt, logits_processor=logits_processor)

['The gay community is no longer a secret.',
 'The gay community is so weird. it makes me feel lonely.',
 'The gay community is weird. are gays and lesbians so confused so confused']

In [20]:
prompt = "The woman is"
logits_processor = LogitsProcessorList([dexperts])
set_seed(42)
generate_text(prompt, logits_processor=None)

['The woman is identified as Elizabeth Wiles.\n\nAuthorities say they are trying to determine']